# Install library

In [1]:
!pip install panda lxml bs4 httpx backtesting yfinance pandas_ta scipy rich

In [2]:
# import library
import pandas as pd
import numpy as np
from datetime import datetime

# Install library

In [3]:
# import library
import pandas as pd
import numpy as np
from datetime import datetime

In [4]:
import concurrent.futures
from datetime import datetime
import glob
import warnings
from rich.progress import track
warnings.filterwarnings("ignore")
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas_ta as taPanda
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from collections import Counter
import httpx

# Utilities Function

In [5]:
def make_api_request(api_endpoint, params):
    with httpx.Client() as client:
        # Make the GET request to the API
        response = client.get(api_endpoint, params=params)
        if response.status_code == 200:
            return response.json()
        print("Error: Failed to retrieve data from API")
        return None

In [6]:
BASE_URL_FMP = "https://financialmodelingprep.com/api/v3"
BASE_URL_BINANCE = "https://fapi.binance.com/fapi/v1/"

FMP_API_KEY = "17c09553207c6d6e7bab10003e604aa8"
BINANCE_API_KEY = ""

In [7]:
def get_historical_price_full_crypto(symbol):
    api_endpoint = f"{BASE_URL_FMP}/historical-price-full/crypto/{symbol}"
    params = {"apikey": FMP_API_KEY}
    return make_api_request(api_endpoint, params)


def get_historical_price_full_stock(symbol):
    api_endpoint = f"{BASE_URL_FMP}/historical-price-full/{symbol}"
    params = {"apikey": FMP_API_KEY}

    return make_api_request(api_endpoint, params)

def get_SP500():
    api_endpoint = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    data = pd.read_html(api_endpoint)
    return list(data[0]['Symbol'])

def get_all_crypto():
    """
    All possible crypto symbols
    """
    return [
        "BTCUSD",
        "ETHUSD",
        "LTCUSD",
        "BCHUSD",
        "XRPUSD",
        "EOSUSD",
        "XLMUSD",
        "TRXUSD",
        "ETCUSD",
        "DASHUSD",
        "ZECUSD",
        "XTZUSD",
        "XMRUSD",
        "ADAUSD",
        "NEOUSD",
        "XEMUSD",
        "VETUSD",
        "DOGEUSD",
        "OMGUSD",
        "ZRXUSD",
        "BATUSD"
        "USDTUSD",
        "LINKUSD",
        "BTTUSD",
        "BNBUSD",
        "ONTUSD",
        "QTUMUSD",
        "ALGOUSD",
        "ZILUSD",
        "ICXUSD",
        "KNCUSD",
        "ZENUSD",
        "THETAUSD",
        "IOSTUSD",
        "ATOMUSD",
        "MKRUSD",
        "COMPUSD",
        "YFIUSD",
        "SUSHIUSD",
        "SNXUSD",
        "UMAUSD",
        "BALUSD",
        "AAVEUSD",
        "UNIUSD",
        "RENBTCUSD",
        "RENUSD",
        "CRVUSD",
        "SXPUSD",
        "KSMUSD",
        "OXTUSD",
        "DGBUSD",
        "LRCUSD",
        "WAVESUSD",
        "NMRUSD",
        "STORJUSD",
        "KAVAUSD",
        "RLCUSD",
        "BANDUSD",
        "SCUSD",
        "ENJUSD",
    ]

def get_financial_statements_lists():
    api_endpoint = f"{BASE_URL_FMP}/financial-statement-symbol-lists"
    params = {"apikey": FMP_API_KEY}
    return make_api_request(api_endpoint, params)

# Split the Data into Train and Test

In [8]:
stock_symbol = "AAPL"
stock_prices = get_historical_price_full_stock(stock_symbol)

data = pd.DataFrame(stock_prices['historical'])

In [9]:
data

,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime
0,2024-01-26,194.27,194.7600,192.2700,192.455,192.455,24879152,24879152,-1.81,-0.934267,193.16,"January 26, 24",-0.009343
1,2024-01-25,195.22,196.2675,193.1125,194.170,194.170,54600287,54460179,-1.05,-0.537850,194.74,"January 25, 24",-0.005378
2,2024-01-24,195.42,196.3800,194.3400,194.500,194.500,53631316,53631300,-0.92,-0.470780,195.21,"January 24, 24",-0.004708
3,2024-01-23,195.02,195.7500,193.8299,195.180,195.180,42355590,42355600,0.16,0.082043,194.82,"January 23, 24",0.000820
4,2024-01-22,192.30,195.3300,192.2600,193.890,193.890,60131852,60133900,1.59,0.826830,193.99,"January 22, 24",0.008268
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,2019-02-01,41.74,42.2400,41.4800,41.630,39.960,130672400,130672400,-0.11,-0.263540,41.79,"February 01, 19",-0.002635
1255,2019-01-31,41.53,42.2500,41.1400,41.610,39.940,162958400,162958400,0.08,0.192630,41.67,"January 31, 19",0.001926
1256,2019-01-30,40.81,41.5400,40.0600,41.310,39.650,244439200,244439200,0.50,1.230000,40.97,"January 30, 19",0.012300
1257,2019-01-29,39.06,39.5300,38.5300,38.670,37.120,166348800,166348800,-0.39,-0.998460,38.91,"January 29, 19",-0.009985


In [10]:
january_2023_index = data[(data['date'] < '2023-01-01')].index
january_2023_index

Index([ 268,  269,  270,  271,  272,  273,  274,  275,  276,  277,
       ...
       1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258],
      dtype='int64', length=991)

In [11]:
prices_before_january_2023 = data.loc[january_2023_index]
prices_before_january_2023

# Assuming prices_before_january_2023 is your DataFrame
prices_before_january_2023 = prices_before_january_2023.rename(columns={
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'volume': 'Volume'  # Only if you have a volume column
})

# Drop any additional columns that are not required
required_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
prices_before_january_2023 = prices_before_january_2023[required_columns]

# Convert the index to a datetime index
prices_before_january_2023.index = pd.to_datetime(prices_before_january_2023.index)

prices_before_january_2023

,Open,High,Low,Close,Volume
1970-01-01 00:00:00.000000268,128.41,129.95,127.43,129.93,77034200
1970-01-01 00:00:00.000000269,127.99,130.48,127.73,129.61,75703700
1970-01-01 00:00:00.000000270,129.67,131.03,125.87,126.04,85438400
1970-01-01 00:00:00.000000271,131.38,131.41,128.72,130.03,69007800
1970-01-01 00:00:00.000000272,130.92,132.42,129.64,131.86,63814900
...,...,...,...,...,...
1970-01-01 00:00:00.000001254,41.74,42.24,41.48,41.63,130672400
1970-01-01 00:00:00.000001255,41.53,42.25,41.14,41.61,162958400
1970-01-01 00:00:00.000001256,40.81,41.54,40.06,41.31,244439200
1970-01-01 00:00:00.000001257,39.06,39.53,38.53,38.67,166348800


In [98]:
prices_after_january_2023 = data.drop(january_2023_index)
prices_after_january_2023

# Assuming prices_after_january_2023 is your DataFrame
prices_after_january_2023 = prices_after_january_2023.rename(columns={
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'volume': 'Volume'  # Only if you have a volume column
})

# Drop any additional columns that are not required
required_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
prices_after_january_2023 = prices_after_january_2023[required_columns]

# Convert the index to a datetime index
prices_after_january_2023.index = pd.to_datetime(prices_after_january_2023.index)


prices_after_january_2023

,Open,High,Low,Close,Volume
1970-01-01 00:00:00.000000000,194.27,194.7600,192.2700,192.455,24879152
1970-01-01 00:00:00.000000001,195.22,196.2675,193.1125,194.170,54600287
1970-01-01 00:00:00.000000002,195.42,196.3800,194.3400,194.500,53631316
1970-01-01 00:00:00.000000003,195.02,195.7500,193.8299,195.180,42355590
1970-01-01 00:00:00.000000004,192.30,195.3300,192.2600,193.890,60131852
...,...,...,...,...,...
1970-01-01 00:00:00.000000263,130.47,133.4100,129.8900,130.150,70790800
1970-01-01 00:00:00.000000264,126.01,130.2900,124.8900,129.620,87754700
1970-01-01 00:00:00.000000265,127.13,127.7700,124.7600,125.020,80962700
1970-01-01 00:00:00.000000266,126.89,128.6600,125.0800,126.360,89113600


# Monte Carlo Simulation

In [99]:
def monte_carlo_simulation(data, days, iterations):
    # Convert to numpy array if data is a pandas Series
    if isinstance(data, pd.Series):
        data = data.to_numpy()

    # Ensure data is a numpy array
    if not isinstance(data, np.ndarray):
        raise TypeError("Data must be a numpy array or pandas Series")

    # Calculate log returns
    log_returns = np.log(data[1:] / data[:-1])
    mean = np.mean(log_returns)
    variance = np.var(log_returns)
    drift = mean - (0.5 * variance)
    daily_volatility = np.std(log_returns)

    future_prices = np.zeros((days, iterations))
    current_price = data[-1]
    for t in range(days):
        shocks = drift + daily_volatility * norm.ppf(np.random.rand(iterations))
        future_prices[t] = current_price * np.exp(shocks)
        current_price = future_prices[t]
    return future_prices


# Markov Chain Simulation Transition Matrix

In [100]:
bins = [0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 200, np.inf]
strata = pd.cut(prices_before_january_2023['Close'], bins=bins)

transition_matrix = pd.crosstab(pd.Series(strata[:-1], name='Current'),
                                pd.Series(strata[1:], name='Next'),
                                normalize='index')

# Markov Chain Simulation

In [101]:
def markov_chain_simulation(transition_matrix, start_state, days):
    states = np.arange(len(transition_matrix))
    current_state = start_state
    state_sequence = [current_state]

    for _ in range(days):
        current_state = np.random.choice(states, p=transition_matrix[current_state])
        state_sequence.append(current_state)

    return state_sequence

# Visualize

In [102]:

simulation_days = 10
mc_iterations = 100
start_state = strata[-1]
mc_prices = monte_carlo_simulation(stock_prices, simulation_days, mc_iterations)
markov_states = markov_chain_simulation(transition_matrix, start_state, simulation_days)

# Visualizing the Monte Carlo simulation
plt.figure(figsize=(10, 6))
for i in range(mc_iterations):
    plt.plot(mc_prices[:, i], linewidth=1)
plt.title('Monte Carlo Simulation of Stock Prices')
plt.xlabel('Days')
plt.ylabel('Price')
plt.show()

TypeError: Data must be a numpy array or pandas Series

#  MCMCStrategy

In [80]:
class MCMCStrategy(Strategy):
    stop_loss_percentage = 0.05  # Dynamic based on volatility
    take_profit_percentage = 0.10  # Dynamic based on Monte Carlo prediction

    def init(self):
        # Initializing indicators and simulations
        self.is_long = False
        self.mc_iterations = 1000
        self.start_state = 0

        self.mc_prices = monte_carlo_simulation(self.data.Close, len(self.data), self.mc_iterations)
        self.markov_states = markov_chain_simulation(transition_matrix, self.start_state, len(self.data))

    def next(self):
        if not self.position:
            # Buy condition: Bullish state and EMA(20) above EMA(50)
            if self.markov_states[-1] == 0:
                self.buy()
                self.is_long = True
                self.entry_price = self.data.Close[-1]
        else:
            if self.is_long:
                # Dynamic stop-loss and take-profit
                stop_loss = self.entry_price * (1 - self.stop_loss_percentage)
                take_profit = self.mc_prices[-1, np.random.randint(self.mc_iterations)]

                if self.data.Close[-1] <= stop_loss or self.data.Close[-1] >= take_profit:
                    self.sell()
                    self.is_long = False

# Create two separate Backtest instances for before and after January 2023

In [81]:
bt_mcmc_before_january = Backtest(prices_before_january_2023, MCMCStrategy,
                             cash=10000, commission=.002,
                             exclusive_orders=True)

bt_mcmc_after_january = Backtest(prices_after_january_2023, MCMCStrategy,
                            cash=10000, commission=.002,
                            exclusive_orders=True)

In [82]:
output_mcmc_before_january = bt_mcmc_before_january.run()

In [46]:
output_mcmc_before_january

Start                     1970-01-01 00:00...
End                       1970-01-01 00:00...
Duration                  0 days 00:00:00....
Exposure Time [%]                   94.651867
Equity Final [$]                   13837.0569
Equity Peak [$]                   17530.53854
Return [%]                          38.370569
Buy & Hold Return [%]              -69.922266
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -52.491689
Avg. Drawdown [%]                   -6.712795
Max. Drawdown Duration    0 days 00:00:00....
Avg. Drawdown Duration    0 days 00:00:00....
# Trades                                  177
Win Rate [%]                        53.107345
Best Trade [%]                      14.540413
Worst Trade [%]                    -16.966703
Avg. Trade [%]                    

In [21]:
output_mcmc_after_january = bt_mcmc_after_january.run()

In [ ]:
output_mcmc_after_january

In [30]:
bt_mcmc_after_january.plot()

RuntimeError: First issue `backtest.run()` to obtain results.

# Run BUYHOLD backtests

In [ ]:
class BuyAndHoldStrategy(Strategy):
    def init(self):
        self.is_long = False

    def next(self):
        # Buy and Hold Strategy
        if not self.is_long:
            self.buy()
            self.is_long = True

In [ ]:

bt_before_january = Backtest(prices_before_january_2023, BuyAndHoldStrategy, cash=10000, commission=.002, exclusive_orders=True)
bt_after_january = Backtest(prices_after_january_2023, BuyAndHoldStrategy, cash=10000, commission=.002, exclusive_orders=True)

In [ ]:
before_after_january = bt_after_january.run()

In [ ]:
output_after_january = bt_after_january.run()

## Access performance metrics for BUYHOLD backtests

In [ ]:

print("Performance Metrics Before January 2023:")
print(output_before_january)
print("\nPerformance Metrics After January 2023:")
print(output_after_january)

In [ ]:
# Define a function to run a backtest and plot the results
def run_backtest_and_plot(prices, strategy_class, title):
    bt = Backtest(prices, strategy_class, cash=10000, commission=.002, exclusive_orders=True)
    output = bt.run()
    bt.plot(title=title)
    return output

In [ ]:
def buy_and_hold_backtest(prices):
    simulation_days = len(prices)
    mc_iterations = 100
    start_state = 0  # Initial state (can be changed)
    mc_prices = monte_carlo_simulation(prices, simulation_days, mc_iterations)
    markov_states = markov_chain_simulation(transition_matrix, start_state, simulation_days)

    # Simple "Buy and Hold" Strategy Backtest
    final_prices = mc_prices[-1, :]  # Prices at the end of the simulation
    initial_price = prices[-1]  # Initial price
    portfolio_values = final_prices - initial_price  # Portfolio values

    # Calculate strategy returns
    strategy_returns = portfolio_values / initial_price
    return strategy_returns

def run_backtest_and_plot(prices, strategy_class, title):
    bt = Backtest(prices, strategy_class, cash=10000, commission=.002, exclusive_orders=True)
    output = bt.run()
    bt.plot(title=title)
    return output

# Perform "Buy and Hold" backtest for prices before January 2023
strategy_returns_before_january = buy_and_hold_backtest(prices_before_january_2023)

# Perform "Buy and Hold" backtest for prices after January 2023
strategy_returns_after_january = buy_and_hold_backtest(prices_after_january_2023)

# Visualize the results
plt.figure(figsize=(10, 6))
plt.plot(strategy_returns_before_january, label="Before January 2023")
plt.plot(strategy_returns_after_january, label="After January 2023")
plt.title('Buy and Hold Strategy Returns')
plt.xlabel('Simulation Iterations')
plt.ylabel('Returns')
plt.legend()
plt.show()

# Run and plot the MCMC-based backtests
output_before_january_mcmc = run_backtest_and_plot(prices_before_january_2023, MCMCStrategy, "MCMC Strategy Before January 2023")
output_after_january_mcmc = run_backtest_and_plot(prices_after_january_2023, MCMCStrategy, "MCMC Strategy After January 2023")

# Access performance metrics for both MCMC-based backtests
print("Performance Metrics for MCMC Strategy Before January 2023:")
print(output_before_january_mcmc)
print("\nPerformance Metrics for MCMC Strategy After January 2023:")
print(output_after_january_mcmc)